## Abstract
In particular, ciphers designed following the **wide-trail strategy** use the branch number of the linear layer to derive bounds on the probability of linear and differential trails.

At FSE 2014, the **LS-design** construction was introduced as a simple and regular structure to design bitsliced block ciphers. It considers the internal state as a bit matrix, and applies alternatively an identical S-Box on all the columns, and an identical L-Box on all the lines.

We study the construction of bitsliced linear transformations with **efficient implementations** using XORs and rotations (optimized for bitsliced ciphers implemented on 32-bit processors), and a **high branch number**.

Our main result is a linear layer for 128-bit ciphers with branch number 21, improving upon the best 32-bit transformation with branch number 12, and the one of Spook with branch number 16.

## 1 Introduction
### 1.1 Design strategies for SPN ciphers

We consider an SPN cipher with $\ell$ parallel SBoxes, each operating on $s$ bits; the block size is $n = \ell s$ (see Figure 1). In this paper, we denote $\mathbb{B}_s$ the set of all $s$-bit elements: $B_s = \{0, 1\}^s$. The internal state is considered as an element in $B^{\ell}_s = (\{0, 1\}^s)^{\ell}$, i.e., a sequence of $\ell$ $s$-bit elements $x = (x_0, x_1, \dots, x_{\ell−1})$. We define the Hamming weight of $x$ as $|x| = \#\{i \in \{0, 1, \dots , \ell − 1\} : x_i \neq 0\}$(Hamming weight over $\mathbb{B}^{\ell}_s$).

#### 1.1.1 The Wide-Trail Strategy

The differential branch number of a linear transformation $\Lambda$ is defined as
$$
\cal B_d(\Lambda) = \text{min}_{x\neq 0}(|\Lambda(x)|+|x|).
$$
This is an important security property, measuring the diffusion of a linear layer: any non-trivial differential characteristics in two consecutive rounds has at least $\cal B_d(\Lambda)$ active SBoxes [DR01, Theorem 1]. Therefore, if the SBox has differential uniformity $\delta$, the probability of any $r$-round differential trail is at most $(\delta/2^s)^{\cal B_d\cdot\lfloor r/2 \rfloor}$.

Similarly, the linear branch number is defined as

$$\cal B_l(\Lambda) = \text{min}_{x\neq 0}(|\Lambda^{\top}(x)|+|x|),$$

where $\Lambda^{\top}$ is the linear transformation whose matrix representation over $\mathbb{F}_2$ is the transpose of the matrix representation of $\Lambda$ (note that we must consider the matrix representation as an $\ell s \times \ell s$ matrix over $\mathbb{F}_2$, not as an $\ell \times \ell$ matrix over $\mathbb{F}_{2^s}$ ). Any non-trivial linear trail in two consecutive rounds has at least $\cal B_l(\Lambda)$ active SBoxes. Therefore, if the SBox has linearity $\lambda$, the expected squared correlation of any $r$-round linear trail is at most $(\lambda^2/2^{2s})^{\cal B_l\cdot \lfloor r/2 \rfloor} =(\lambda/2^{s})^{2\cal B_l\cdot \lfloor r/2 \rfloor}$.

We have $2 \leq \cal B_d(\Lambda) \leq \ell + 1$ and $2 \leq \cal B_l(\Lambda) \leq \ell + 1$. A linear layer is called an MDS matrix when $\cal B_d(\Lambda) = \ell + 1$; this condition is equivalent to $\cal B_l(\Lambda) = \ell + 1$. Besides, when the linear layer is orthogonal (i.e. when $\Lambda^{\top} = \Lambda^{-1}$), we have $\cal B_l(\Lambda) = \cal B_d(\Lambda)$.